In [2]:
import geopandas as gpd
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import h5py
import sys


sys.path.append('../../')
sns.set_style('whitegrid')

from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
from src.data.data_utils import *

In [1]:
data_root = '/scratch/bbug/ayang1/raw_data/lucas/s1_lucas_2018'
save_path = '/scratch/bbug/ayang1/datasets/lucas_dataset_ieitptnl'

# Collect ee generated time series data
ds = [
    'S1_point_10days_10m_1Jan-31Dec_Ireland_ratio-db.csv',
    'S1_point_10days_10m_1Jan-31Dec_Italy_ratio-db.csv',
    'S1_point_10days_10m_1Jan-31Dec_Netherlands_ratio-db.csv',
    'S1_point_10days_10m_1Jan-31Dec_Portugal_ratio-db.csv'      
]

In [3]:
# Lucas labels 
labels = pd.read_csv('/scratch/bbug/ayang1/raw_data/lucas/lucas_2018/copernicus_filtered/lucas_2018_filtered.csv')

dfs = []
for path in ds:
    dfs.append(pd.read_csv(os.path.join(data_root, path)))

data= pd.concat(dfs)

/tmp/ipykernel_2280934/4214113462.py:2: DtypeWarning: Columns (34,39,44) have mixed types. Specify dtype option on import or set low_memory=False.
  labels = pd.read_csv('/scratch/bbug/ayang1/raw_data/lucas/lucas_2018/copernicus_filtered/lucas_2018_filtered.csv')


In [4]:
# Add labels to signals based on point id
crop_data = add_lucas_labels(data, labels)
crop_data.drop('system:index', axis=1, inplace=True)
crop_data = crop_data.loc[crop_data['LABEL']!='NOT_CROP']

Loading files
Creating dataset of size 239980


100%|██████████| 239980/239980 [01:09<00:00, 3440.78it/s]


In [5]:
crop_data.to_csv(os.path.join(save_path, 'IE_IT_PT_NL_s1.csv'))
crop_data = pd.read_csv(os.path.join(save_path, 'IE_IT_PT_NL_s1.csv')).drop('Unnamed: 0', axis=1)
crop_data = crop_data.drop(['POINT_ID', '.geo'], axis=1)

In [10]:
# Filter out classes with less than 1000 samples
counts = np.unique(crop_data['LABEL'], return_counts=True)
to_drop = [counts[0][i] for i in range(len(counts[0])) if counts[1][i] < 1000]
crop_data = crop_data.loc[~crop_data['LABEL'].isin(to_drop)]

print('Dropped classes:', to_drop)

Dropped classes: ['B14', 'B17', 'B18', 'B19', 'B22', 'B23', 'B31', 'B32', 'B33', 'B35', 'B36', 'B37', 'B42', 'B43', 'B44', 'B51', 'B71', 'B74', 'B75', 'B81', 'B82', 'B83']


In [11]:
# Save datasets as npy files
arr = crop_data.to_numpy()
vhvv = arr[:, np.newaxis, 0:36]
vh = arr[:, np.newaxis, 36:72]
vv = arr[:, np.newaxis, 72:108]
labels = arr[:, 108]

data = np.concatenate([vhvv, vh, vv], axis=1)

In [13]:
train_signals, test_signals, train_labels, test_labels = train_test_split(data, labels, test_size=0.3, random_state=42)

In [14]:
np.save(os.path.join(save_path, 'train_signals.npy'), train_signals)
np.save(os.path.join(save_path, 'test_signals.npy'), test_signals)
np.save(os.path.join(save_path, 'train_labels.npy'), train_labels)
np.save(os.path.join(save_path, 'test_labels.npy'), test_labels)

In [15]:
subset = train_signals[15, :, :]